### Load Packages

In [2]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

## Load dataset and clean data - Adam

In [3]:
data = pd.read_csv('Life Expectancy Data.csv')

data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

gdp_per_capita = data.gdp/data.population
data['gdp_per_capita'] = gdp_per_capita

np.random.seed(2)
train = data.sample(round(data.shape[0]*0.8)) # 80% training, 20% test
test = data.drop(train.index)

/var/folders/0x/61vcmsg94cs94n1kc433lq8w0000gn/T/ipykernel_76720/1437335773.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/0x/61vcmsg94cs94n1kc433lq8w0000gn/T/ipykernel_76720/1437335773.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


In [5]:
train.head(1)

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,hepatitis_b,measles,...,total_expenditure,diphtheria,hiv/aids,gdp,population,thinness__1-19_years,thinness_5-9_years,income_composition_of_resources,schooling,gdp_per_capita
1576,Malawi,2009,Developing,51.5,491.0,36,1.16,7.915071,93.0,21,...,1.18,93.0,14.9,42.737966,1471462.0,6.9,6.8,0.415,9.9,0.000029


## Variable Selection/VIF - Judd

## Final Regression Analysis - Alissa